<a href="https://colab.research.google.com/github/Tuevu110405/End-to-end-QA/blob/feature%2Fpush_colab/End_to_end_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qq datasets evaluate transformers[sentencepiece]
!pip install -qq accelerate
!apt install git-lfs

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install transformers

In [ ]:
!pip install peft

In [ ]:
import numpy as np
from tqdm.auto import tqdm
import collections

import torch

from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering
from transformers import TrainingArguments
from transformers import Trainer
import evaluate

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


In [ ]:
MODEL_NAME = "distilbert-base-uncased"

MAX_LENGTH = 384

STRIDE = 128

In [ ]:
DATASET_NAME = 'squad_v2'
raw_datasets = load_dataset(DATASET_NAME)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.92k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        sequence_ids = inputs.sequence_ids(i)
        context_start = sequence_ids.index(1)
        context_end = len(sequence_ids) - 1 - sequence_ids[::-1].index(1)  # Tối ưu hóa việc tìm context_end

        answer = answers[sample_idx]
        if not answer['text']:  # Kiểm tra độ dài bằng 0 môt cách Pythonic hơn
            start_positions.append(0)
            end_positions.append(0)
        else:
            start_char = answer["answer_start"][0]
            end_char = start_char + len(answer["text"][0])

            if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
                start_positions.append(0)
                end_positions.append(0)
            else:
                start_positions.append(next((idx - 1 for idx in range(context_start, context_end + 1) if offset[idx][0] > start_char), context_end)) # Sử dụng generator expression và next()
                end_positions.append(next((idx + 1 for idx in range(context_end, context_start - 1, -1) if offset[idx][1] < end_char), context_start)) # Sử dụng generator expression và next()


    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs




In [ ]:
train_dataset = raw_datasets["train"].map(
    preprocess_training_example,
    batched = True,
    remove_columns = raw_datasets["train"].column_names
)

len(raw_datasets["train"]), len(train_dataset)

Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

(130319, 131754)

In [ ]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = [examples["id"][idx] for idx in sample_map]  # Sử dụng list comprehension

    # Điều chỉnh offset_mapping trực tiếp trong vòng lặp, tránh truy cập nhiều lần
    for i in range(len(inputs["input_ids"])):
        sequence_ids = inputs.sequence_ids(i)
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None
            for k, o in enumerate(inputs["offset_mapping"][i])
        ]

    inputs["example_id"] = example_ids
    return inputs

In [ ]:
validation_dataset = raw_datasets["validation"].map(
    preprocess_validation_examples,
    batched = True,
    remove_columns = raw_datasets["validation"].column_names
)

len(raw_datasets["validation"], len(validation_dataset))


TypeError: len() takes exactly one argument (2 given)

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)

args = TrainingArguments(
    output_dir = "distill-finetuned-squadv2",
    evaluation_strategy = "no",
    save_strategy = "epoch",
    learning_rate = 2e-5,
    num_train_epochs = 3,
    weight_decay = 0.01,
    fp16 = True,
    push_to_hub = True,
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


<ipython-input-32-3189b18931a3>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


RuntimeError: WandbCallback requires wandb to be installed. Run `pip install wandb`.

In [ ]:


import os
os.environ["WANDB_DISABLED"] = "true"

args = TrainingArguments(
    output_dir="distill-finetuned-squadv2",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    push_to_hub=True,
    report_to="none"
)



/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
#Fine tuning LLM
trainer = Trainer(
    model = model,
    args=args,
    train_dataset = train_dataset,
    eval_dataset = validation_dataset,
    tokenizer = tokenizer,
)

trainer.train()

<ipython-input-34-3189b18931a3>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,3.233700
1000,2.340000
1500,2.029800
2000,1.859200
2500,1.749800
3000,1.687500
3500,1.609000
4000,1.587400
4500,1.555300
5000,1.557400


TrainOutput(global_step=49410, training_loss=1.042436260570039, metrics={'train_runtime': 4777.9373, 'train_samples_per_second': 82.726, 'train_steps_per_second': 10.341, 'total_flos': 3.873165421863629e+16, 'train_loss': 1.042436260570039, 'epoch': 3.0})

In [ ]:
# Gửi dữ liệu đào tạo lên Hub
trainer.push_to_hub(commit_message="Training complete")

# Tải metric "squad" từ thư viện evaluate
metric = evaluate.load("squad_v2")

No files have been modified since last commit. Skipping to prevent empty commit.


In [ ]:
N_BEST = 20
MAX_ANS_LENGTH = 30

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature['example_id']].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example['id']
        context = example['context']
        answers = []

        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]['offset_mapping']

            start_indexes = np.argsort(start_logit)[-1:-N_BEST - 1:-1].tolist()
            end_indexes = np.argsort(end_logit)[-1:-N_BEST - 1:-1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    if end_index < start_index or end_index - start_index + 1 > MAX_ANS_LENGTH:  # Kết hợp hai điều kiện
                        continue

                    answers.append({
                        'text': context[offsets[start_index][0]:offsets[end_index][1]],
                        'logit_score': start_logit[start_index] + end_logit[end_index],
                    })

        if answers:
            best_answer = max(answers, key=lambda x: x['logit_score'])
            predicted_answers.append({
                'id': example_id,
                'prediction_text': best_answer['text'],
                'no_answer_probability': 1 - best_answer['logit_score'],
            })
        else:
            predicted_answers.append({
                'id': example_id,
                'prediction_text': '',
                'no_answer_probability': 1.0,
            })

    theoretical_answers = [{'id': ex['id'], 'answers': ex['answers']} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)



predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
results = compute_metrics(start_logits, end_logits, validation_dataset, raw_datasets["validation"])
print(results)

  0%|          | 0/11873 [00:00<?, ?it/s]

{'exact': 42.75246357281226, 'f1': 47.51293725623134, 'total': 11873, 'HasAns_exact': 72.30094466936572, 'HasAns_f1': 81.83554386694244, 'HasAns_total': 5928, 'NoAns_exact': 13.288477712363331, 'NoAns_f1': 13.288477712363331, 'NoAns_total': 5945, 'best_exact': 67.01760296470985, 'best_exact_thresh': -10.328125, 'best_f1': 69.07801227762936, 'best_f1_thresh': -10.20703125}


In [ ]:
from transformers import pipeline

PIPELINE_NAME = 'question-answering'
MODEL_NAME = 'tientuevu/distilbert-finetuned-squadv2'
pipe = pipeline(PIPELINE_NAME, model = MODEL_NAME)

config.json:   0%|          | 0.00/561 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
INPUT_QUESTION = "What is my name?"
INPUT_CONTEXT = "My name is AI Vietnam and I live in Vietnam"
pipe(question = INPUT_QUESTION, context = INPUT_CONTEXT)

{'score': 0.9704906344413757, 'start': 11, 'end': 21, 'answer': 'AI Vietnam'}

RETRIEVER FAISS

In [ ]:
!pip install -qq transformers[sentencepiece]==4.35.2 datasets==2.16.1 evaluate==0.4.1
!sudo apt-get install libomp-dev
!pip install -qq faiss-gpu

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libomp-dev is already the newest version (1:14.0-55~exp2).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [ ]:


!pip install -qq faiss-gpu


ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [ ]:
!pip install faiss-cpu
!nvcc --version


!pip install --no-cache-dir "faiss-gpu==1.7.2+cu118"

!pip install --no-cache-dir faiss-gpu


!conda install -c pytorch -c nvidia -c conda-forge faiss-gpu

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
ERROR: Could not find a version that satisfies the requirement faiss-gpu==1.7.2+cu118 (from versions: none)
ERROR: No matching distribution found for faiss-gpu==1.7.2+cu118
ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu
/bin/bash: line 1: conda: command not found


In [ ]:
!pip uninstall transformers peft -y

Found existing installation: transformers 4.35.2
Uninstalling transformers-4.35.2:
  Successfully uninstalled transformers-4.35.2
Found existing installation: peft 0.14.0
Uninstalling peft-0.14.0:
  Successfully uninstalled peft-0.14.0


In [ ]:
!pip install transformers

In [ ]:

import numpy as np
import collections
import torch
import faiss
import evaluate

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
from transformers import AutoModelForQuestionAnswering
from transformers import TrainingArguments
from transformers import Trainer
from tqdm.auto import tqdm

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
DATASET_NAME = 'squad_v2'
raw_datasets = load_dataset(DATASET_NAME, split='train+validation')
raw_datasets

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 142192
})

In [ ]:
raw_datasets = raw_datasets.filter(
    lambda x: len(x['answers']['text']) > 0
)

Filter:   0%|          | 0/142192 [00:00<?, ? examples/s]

In [ ]:
MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(device)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [ ]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list,
        padding = True,
        truncation = True,
        return_tensors = "pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [ ]:
EMBEDDING_COLUMN = 'question_embedding'
embeddings_dataset = raw_datasets.map(
    lambda x: {
        EMBEDDING_COLUMN: get_embeddings(
            x['question']
        ).detach().cpu().numpy()[0]
    }
)

Map:   0%|          | 0/92749 [00:00<?, ? examples/s]

In [ ]:
embeddings_dataset.add_faiss_index(column=EMBEDDING_COLUMN)

  0%|          | 0/93 [00:00<?, ?it/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'question_embedding'],
    num_rows: 92749
})

In [ ]:
input_question = 'When did Beyonce start becoming popular?'

input_quest_embedding = get_embeddings([input_question])
input_quest_embedding = input_quest_embedding.cpu().detach().numpy()

TOP_K = 5
scores, samples = embeddings_dataset.get_nearest_examples(
    EMBEDDING_COLUMN, input_quest_embedding, k=TOP_K
)

for idx, score in enumerate(scores):
    print(f'Top {idx + 1}\tScore: {score}')
    print(f'Question: {samples["question"][idx]}')
    print(f'Context: {samples["context"][idx]}')
    print()

Top 1	Score: 0.0
Question: When did Beyonce start becoming popular?
Context: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".

Top 2	Score: 2.613530397415161
Question: When did Beyoncé rise to fame?
Context: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress.

In [ ]:
from transformers import pipeline

PIPELINE_NAME = 'question-answering'
MODEL_NAME = 'thangduong0509/distilbert-finetuned-squadv2'
pipe = pipeline(PIPELINE_NAME, model=MODEL_NAME)

config.json:   0%|          | 0.00/561 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
print(f'Input question: {input_question}')
for idx, score in enumerate(scores):
    question = samples["question"][idx]
    context = samples["context"][idx]
    answer = pipe(
        question=question,
        context=context
    )
    print(f'Top {idx + 1}\tScore: {score}')
    print(f'Context: {context}')
    print(f'Answer: {answer}')
    print()

Input question: When did Beyonce start becoming popular?
Top 1	Score: 0.0
Context: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".
Answer: {'score': 0.6091992855072021, 'start': 276, 'end': 286, 'answer': 'late 1990s'}

Top 2	Score: 2.613530397415161
Context: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an Amer